# Examen Práctico: Clasificación de dígitos con Redes Neuronales Densas (Keras)

**Instrucciones generales**:

- Trabaja en este cuaderno y **completa únicamente las celdas de código** indicadas (no borres las celdas de enunciado).
- Este examen pide que implementes un clasificador **multiclase** usando **redes densas** (no usar convoluciones).
- Aunque los CNNs suelen rendir mejor en imágenes, **NO** usarás convoluciones aquí (esto es parte del enunciado).
- Usa `TensorFlow / Keras` para construir, entrenar y evaluar el modelo.
- Anota tus respuestas y observaciones donde se indique.

![](https://www.tensorflow.org/static/datasets/overview_files/output_DpE2FD56cSQR_1.png)
---

**Formato del examen**: cada sección contiene
1. Un enunciado (qué debes hacer), con **pistas** y la forma esperada de los datos.
2. Una celda de código vacía donde debes implementar la solución.

Al final debes entregar el cuaderno con las celdas completadas y una breve conclusión (máx. 300 palabras) sobre por qué un CNN funcionaría mejor.


## Sección 1 — Carga de librerías y comprobación de versión

**Objetivo**: Importar las librerías necesarias y comprobar la versión de Python/TensorFlow.

**Debes hacer**:
- Importar `numpy`, `matplotlib.pyplot`, `tensorflow` (alias `tf`) y `sklearn.metrics` (para reporte y matriz de confusión).
- Mostrar `python --version` o `sys.version` y `tf.__version__`.

**Pista**: usa `from tensorflow import keras` y `from tensorflow.keras.utils import to_categorical`.


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
import math

In [ ]:
# de Version de Tensorflow y si usa GPU
print("TensorFlow:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))


## Sección 2 — Carga del dataset

**Objetivo**: Cargar el dataset `mnist` de `tf.keras.datasets`.

**Debes hacer**:
- Cargar `x_train, y_train, x_test, y_test` usando `tf.keras.datasets.mnist.load_data()`.
- Mostrar las formas de los arrays y un resumen de los valores (mínimo, máximo) de `x_train` y `y_train`.

**Forma esperada**:
- `x_train` → (60000, 28, 28)
- `y_train` → (60000,)

**Pista**: imprime `x_train.shape, y_train.shape, x_train.min(), x_train.max()`.


In [ ]:
#Cargue de datos tfds.load()
datos, metadatos = tfds.load('mnist', as_supervised=True, with_info=True)

In [ ]:
#Mostrar la vriable datos


In [ ]:
#Mostrar la vriable metadatos


<font color="green"> **Analisis**: Que tiene esa variable? </font>

#**Visualización de una imagen en MNIST**

Obtén el conjunto de **entrenamiento **del dataset mnist.

Selecciona la primera imagen y su etiqueta usando .take(1).

Convierte la imagen a numpy y guarda la etiqueta en una variable.

**Imprime en pantalla:**

La etiqueta de la imagen.

La forma de la imagen.

La primera fila de la matriz de píxeles.

Muestra la imagen en blanco y negro usando matplotlib.pyplot.imshow() con cmap=plt.cm.binary.

*Forma esperada:*

La etiqueta debe ser un número entre 0–9.

La forma de la imagen debe ser (28, 28)..

In [ ]:
# Obtener el conjunto de entrenamiento


# Tomar el primer elemento (imagen, etiqueta)







**El objeto datos_entrenamiento es un tf.data.Dataset**

Para conocer las dimensiones de las imágenes y etiquetas, debemos agrupar el dataset en lotes y luego inspeccionarlos.

Agrupa el dataset de entrenamiento en lotes de tamaño 32 usando .batch(32).

Toma un solo batch con .take(1).

**Imprime en pantalla:**

La forma del batch de imágenes.

La forma del batch de etiquetas.

**Forma esperada:**

Imágenes → (32, 28, 28)

Etiquetas → (32,)
![](https://raw.githubusercontent.com/adiacla/bigdata/refs/heads/master/batch32.jpg)


In [ ]:
# Para ver un batch(32) solo de ejemplo:





## Sección 3 — Preprocesamiento

**Objetivo**: Preprocesar los datos para una red densa.

Vamos a crear las variables con datos_entrenamiento y datos_pruebas.

In [ ]:
# Definir datasets de entrenamiento y prueba





**Cantidad de ejemplos en los conjuntos de datos**

El objeto metadatos que devuelve tfds.load() contiene información sobre los subconjuntos del dataset.

Calcula:

- Obtén el número total de ejemplos en el conjunto de entrenamiento (train).

- Obtén el número total de ejemplos en el conjunto de prueba (test).

Guarda esos valores en las variables:

In [ ]:
#Número de ejemplos






**Normalización del dataset**

Las imágenes en MNIST tienen valores de píxeles entre 0 y 255. Antes de entrenar una red neuronal es recomendable escalar estos valores al rango [0,1].

Define una función llamada normalizar(imagenes, etiquetas) que:

- Convierta las imágenes a tipo tf.float32.

- Divida cada valor de píxel entre 255.

- Devuelva (imagenes, etiquetas).

- Aplica esta función al dataset de entrenamiento y al de pruebas usando .map().

- Comprueba que los datos fueron transformados correctamente tomando un ejemplo del dataset y mostrando sus valores mínimo y máximo.

**Pista:**

- Usa tf.cast() para convertir el tipo de datos.

Recuerda que .map() aplica la transformación a cada elemento del dataset.

Forma esperada: los valores de los píxeles deben estar entre 0.0 y 1.0.

In [ ]:
#Funcion de normalizacion para los datos (Pasar valor de los pixeles de 0-255 a 0-1)
# Normalización







#**Preparación de los datasets en TensorFlow**

Para entrenar un modelo de Deep Learning de forma eficiente, es necesario preparar los datos con un pipeline que incluya normalización, barajado y creación de lotes.

Define una función normalizar que convierta los valores de las imágenes a flotantes entre 0 y 1.

Recuerda que los datos de las imágenes suelen venir entre 0 y 255.

La función debe recibir (x, y) y retornar (x_normalizado, y).

Declara la constante TAMANO_LOTE con valor 32.

**Prepara el conjunto de *datos_entrenamiento* aplicando en orden:**

.map(normalizar)

.cache()

.shuffle(num_datos_entrenamiento) usando como buffer el total de ejemplos.

.batch(TAMANO_LOTE)

.repeat() para poder iterar en varias épocas.

**Prepara el conjunto de *datos_pruebas* aplicando en orden:**

.map(normalizar)

.cache()

.batch(TAMANO_LOTE)

No uses shuffle() ni repeat() en pruebas.

In [ ]:
# Preparar datasets







**Optimización del dataset con caché**

Una forma de acelerar el entrenamiento en TensorFlow es usar .cache() sobre el dataset. Esto guarda los datos en memoria después de la primera lectura, evitando tener que cargarlos del disco en cada época.

- Aplica .cache() al dataset de entrenamiento (datos_entrenamiento).

- Aplica también .cache() al dataset de pruebas (datos_pruebas).

Explica brevemente en un comentario por qué usar caché puede mejorar la velocidad de entrenamiento.

**Pista: **.cache() se aplica directamente sobre el objeto tf.data.Dataset.

In [ ]:
#Agregar a cache (usar memoria en lugar de disco, entrenamiento mas rapido)






**Definición de clases en MNIST**

El dataset MNIST contiene 10 clases que corresponden a los dígitos escritos a mano: del 0 al 9.

Define una lista llamada clases que contenga las etiquetas de cada dígito como cadenas de texto.

Esta lista se usará más adelante para mostrar el nombre de la clase debajo de cada imagen al graficar.

In [ ]:
#Ejecute esta celda
clases = ["0","1","2","3","4","5","6","7","8","9"]

**Visualización de ejemplos de MNIST**

El dataset MNIST contiene imágenes de dígitos escritos a mano. Para comprobar que la carga y preprocesamiento de datos es correcto, es útil visualizar algunos ejemplos.

- Toma 25 imágenes del conjunto de entrenamiento usando .take(25).

- Dibuja las imágenes en una cuadrícula de 5x5 usando matplotlib.

**Asegúrate de:**

- Convertir cada imagen a una matriz de 28x28.

- Quitar los ejes (xticks, yticks) y la cuadrícula (grid).

- Mostrar cada imagen en escala de grises (cmap=plt.cm.binary).

- Colocar como título (o etiqueta) el número correspondiente a la clase.

Forma esperada: un mosaico de 25 imágenes, cada una mostrando un dígito escrito a mano y su etiqueta correspondiente.


In [ ]:
#Codigo para mostrar imagenes del set, no es necesario ejecutarlo, solo imprime unos numeros :)
# Para ver un batch (32)









## Sección 4 — Construcción del modelo (Keras)

**Objetivo**: Definir una red neuronal densa multiclase en Keras. Sin convoluciones ni dropout, ya que aún no lo hemos estudiado.

**Requisitos mínimos**:
- Usa `tf.keras.Sequential` (o Functional API) con *al menos* **2 capas ocultas**.
- Las capas ocultas deben usar `activation='relu'`.
- La capa de salida debe ser `Dense(10, activation='softmax')`.
- Añadir al menos **una** técnica de regularización (BatchNormalization).
- Pongale nombres a las capas
-Usa add para adicoonar capas

**Sugerencia de arquitectura (no obligatoria)**:
- Input (784) → Dense(512, relu) → Dropout(0.2) → Dense(256, relu) → Dense(10, softmax)


**Pista**: muestra el `model.summary()` al terminar.


In [ ]:
#Cree el Modelo









*Debes además** compilar el modelo con:
- `optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)`
- `loss=tf.keras.losses.SparseCategoricalCrossentropy()  Se usa cuando las etiquetas son enteros (0, 1, 2, ..., 9).
- `metrics=['accuracy']`


In [ ]:
# Compilar el modelo








**Resumen de la arquitectura del modelo**

Una vez que definas tu red neuronal, utiliza el método summary() para mostrar un resumen de la arquitectura del modelo.

Aplica el método .summary() al objeto modelo.

- Observa la salida en pantalla:

- Nombre de cada capa.

- Dimensión de salida de cada capa.

- Cantidad de parámetros entrenables.

In [ ]:
modelo.summary()

**Número de ejemplos en MNIST**

El objeto metadatos que devuelve tfds.load() contiene información sobre los subconjuntos del dataset.

- Obtén el número de ejemplos en el conjunto de entrenamiento (train).

- Obtén el número de ejemplos en el conjunto de pruebas (test).

- Imprime ambos valores en pantalla.

**Forma esperada:**

Entrenamiento → 60000

Pruebas → 10000

**Pista:** usa la propiedad .splits["nombre"].num_examples.

In [ ]:
# Número de ejemplos en train y test




# Imprimir resultados




## Sección 5 — Entrenamiento

**Objetivo**: Entrenar el modelo con validación y callbacks.

**Debes hacer**:
- Entrenar el modelo con `batch_size=32` y `epochs=100`.
- Usa `validation_split=0.01`.
- Implementa los callbacks:
  - `EarlyStopping` con `monitor='val_loss'`, `patience=5`, `restore_best_weights=True`.
- Guarda el historial de entrenamiento en una variable `history` o 'historial'.

**Pista**: `model.fit(x_train, y_train, validation_split=0.1, callbacks=[...])`.


## **Definición del tamaño de lote (batch size)**

En el entrenamiento de redes neuronales, los datos se procesan en lotes (o batches) en lugar de pasar todas las muestras al mismo tiempo.

**Define una variable llamada TAMANO_LOTE.**

Asígnale el valor 32, que será el número de ejemplos que el modelo procesará en cada iteración.

Explica brevemente en un comentario por qué es importante usar lotes en lugar de procesar todos los datos a la vez.

**Pista: **el tamaño de lote es un hiperparámetro que afecta la velocidad de entrenamiento y el uso de memoria.

In [ ]:
#Defina el numero de lotes



**Early Stopping en Keras**

Durante el entrenamiento, un modelo puede dejar de mejorar y comenzar a sobreajustarse. Para evitarlo, se utiliza Early Stopping.

Importa la clase EarlyStopping desde tensorflow.keras.callbacks.

Crea un objeto llamado early_stopping que:

Monitoree la métrica 'val_loss'.

Tenga una paciencia (patience) de 5, lo que significa que el entrenamiento se detendrá si la pérdida de validación no mejora en 5 épocas consecutivas.

Explica en un comentario cómo ayuda Early Stopping a prevenir el sobreajuste.

In [ ]:
# defina la variable early_stopping






**Entrenamiento del modelo con validación**

Ahora que ya has definido y compilado tu red neuronal, es momento de entrenarla.

- Usa el método .fit() del objeto modelo.

*Configura los siguientes parámetros:*

- datos_entrenamiento como dataset de entrada.

- epochs=100 como número máximo de épocas.

- steps_per_epoch=math.ceil(num_datos_entrenamiento / TAMANO_LOTE) para definir el número de pasos por época.

- callbacks=[early_stopping] para detener el entrenamiento si no hay mejora en la pérdida de validación.

- verbose=1 para mostrar la barra de progreso durante el entrenamiento.

- Guarda el resultado en una variable llamada historial.

In [ ]:
# Cree la variable historial y entrene el modelo









## Sección 6 — Evaluación y análisis

**Objetivo**: Evaluar el modelo en el conjunto de prueba y analizar errores.

**Debes hacer**:
1. Cargar (si existe) `best_model.h5` y evaluar en `x_test, y_test`, reportando `test_loss` y `test_accuracy`.
2. Obtener predicciones (probabilidades y clases) sobre `x_test`.
3. Calcular y mostrar la **matriz de confusión** y el **classification_report** usando `sklearn.metrics`.
4. Mostrar 6 ejemplos donde el modelo se equivocó: la imagen, la etiqueta verdadera y la predicha.

**Pista**: para la matriz de confusión usa `confusion_matrix(y_true, y_pred_classes)` (pasar etiquetas no one-hot).


**Valuación del modelo en datos de prueba**

Una vez entrenado tu modelo, evalúalo utilizando el conjunto de datos de prueba.

Usa el método .evaluate() del modelo sobre datos_pruebas.

Define el parámetro steps como math.ceil(num_datos_pruebas / TAMANO_LOTE) para recorrer todo el conjunto.

Guarda los resultados en las variables perdidas y precision.

Imprime en pantalla la precisión con 4 decimales usando:

**Evaluación del modelo en el conjunto de prueba**

Una vez finalizado el entrenamiento, es necesario evaluar el desempeño real del modelo sobre datos que no ha visto antes.

- Usa el método .evaluate() sobre datos_pruebas.

- Configura el parámetro steps como math.ceil(num_datos_pruebas / TAMANO_LOTE) para recorrer todas las muestras del conjunto.

- Guarda los valores de salida en las variables perdidas y precision.

- Muestra en consola la precisión obtenida con cuatro decimales utilizando:

In [ ]:
# Evaluar el modelo con los datos de pruebas



**Guardar el modelo entrenado**

Una vez entrenado y evaluado, guarda tu modelo para poder cargarlo más adelante sin necesidad de volver a entrenarlo.

Utiliza el método .save() de Keras para guardar el modelo en el formato nativo de Keras con la extensión .keras.

In [ ]:
# Guardar el modelo en el formato  de Keras



## Sección 7 — Curvas de entrenamiento

**Objetivo**: Graficar la pérdida y la precisión durante el entrenamiento.

**Debes hacer**:
- Usar `history.history` para graficar:
  - `loss` vs `val_loss`
  - `accuracy` vs `val_accuracy`

**Pista**: usa `matplotlib` y añade títulos, leyenda y ejes correctamente.


In [ ]:

# Obtener los valores del historial
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs_range = range(1, len(acc) + 1)

# Gráfica de accuracy





# Gráfica de pérdida







## Sección 9 — Evaluación del modelo con imágenes externas

Usa las siguientes imágenes de dígitos manuscritos:

Número 3

Número 6

Descarga cada imagen desde la URL.

Conviértela a escala de grises y redimensiona a 28×28 píxeles.

Normaliza los valores de los píxeles (0–255 → 0–1).

Ajusta la forma a (1, 28, 28, 1).

Muestra la imagen procesada y la predicción que hace el modelo.

In [ ]:
#Solo ejecute
import numpy as np
import requests
from PIL import Image
from io import BytesIO

# Función para cargar y preparar una imagen externa
def preparar_imagen(url):
    # Descargar imagen desde la URL
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert("L")  # escala de grises

    # Redimensionar a 28x28
    img = img.resize((28, 28))

    # Convertir a array y normalizar
    img_array = np.array(img) / 255.0

    # Cambiar forma a (1, 28, 28, 1)
    img_array = np.expand_dims(img_array, axis=(0, -1))

    return img_array, img

# URLs de las imágenes
urls = [
    "https://raw.githubusercontent.com/adiacla/bigdata/refs/heads/master/numero3.jpg",
    "https://raw.githubusercontent.com/adiacla/bigdata/refs/heads/master/numero2.jpg"
]

for url in urls:
    procesada, original = preparar_imagen(url)

    # Mostrar imagen original
    plt.imshow(original, cmap="gray")
    plt.title("Imagen original")
    plt.axis("off")
    plt.show()

    # Predicción
    prediccion = modelo.predict(procesada)
    clase_predicha = np.argmax(prediccion)

    print("URL:", url)
    print("El modelo predice que es el número:", clase_predicha)
    print("-" * 50)


---

## Entrega y rúbrica de evaluación

**Rúbrica (100 puntos)**:
- Sección 1: Importaciones y versiones — 5 pts
- Sección 2: Carga de datos y verificación — 10 pts
- Sección 3: Preprocesamiento correcto — 15 pts
- Sección 4: Arquitectura bien definida y compilación — 20 pts
- Sección 5: Entrenamiento correcto con callbacks — 20 pts
- Sección 6: Evaluación y análisis (matriz de confusión + ejemplos) — 20 pts
- Sección 7: Gráficas de entrenamiento — 5 pts
- Sección 8: Respuesta teórica — 5 pts

**Formato de entrega**: sube este notebook con todas las celdas de código completadas y `best_model.h5` (si se generó) en el mismo directorio.

Buena suerte 😊
